*On va regarder comment améliorer la fonction de reward*
-

In [19]:
#1 on se place 
import os
import ipynbname

chemin_notebook = ipynbname.path()
dossier_notebook = os.path.dirname(chemin_notebook)
os.chdir(dossier_notebook)
os.chdir('../..')
print("Répertoire actuel :", os.getcwd())

Répertoire actuel : /Users/dan2/Desktop/Télécom-master-spé/Projets_perso/Deep/Showdown_AI/my_showdown_ai_git


In [20]:
from poke_env.player.random_player import RandomPlayer
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import BaseCallback
import json
from poke_env.environment.pokemon import Pokemon
import poke_env
import pandas as pd
import numpy as np
from poke_env.data import GenData
from poke_env.environment.abstract_battle import AbstractBattle
from poke_env.player.random_player import RandomPlayer
from stable_baselines3.common.env_util import make_vec_env
from IPython.display import display
import ipywidgets as widgets
from poke_env.player import Gen4EnvSinglePlayer
from poke_env.data import GenData
import torch
from gymnasium.spaces import Space, Box, Discrete

In [21]:
from Players.Player5 import *

In [22]:
MAX_LEN_WEATHER = 2
LEN_POKE = 16 
LEN_ACTIVE_POKE = 23 
MAX_LEN_TEAM_SWITCHES = 80 #
IDX_ACTION_SPACE_SIZE = 232
class adapted_obs :
    def __init__(self,battle) :
        self.dict_observation = battle.current_observation.__dict__ 
        #et stocker l'état précédent ? Pour l'instant on passe
        self.side_conditions = np.array(get_side_conditions(self.dict_observation,1),dtype=np.float32) # padding et normalisation déjà réalisés dedans

        self.opponent_side_conditions = np.array(get_side_conditions(self.dict_observation,0),dtype=np.float32) # padding et normalisation déjà réalisés dedans 
        
        self.weather = get_weather(self.dict_observation)
        self.weather[0] = weather_to_id[self.weather[0]]/4
        self.weather = pad_list(self.weather,max_len =MAX_LEN_WEATHER) # padding 
        self.weather = np.array(self.weather,dtype=np.float32)

        self.active_pokemon =np.array( ObservedPokemon_to_list(self.dict_observation["active_pokemon"],is_active_pokemon=1,is_opponent_team=0),dtype=np.float32)
        self.my_team_switches = np.array(get_my_switches_with_all_infos(battle.available_switches,self.dict_observation),dtype=np.float32)
        #Calcul du action space avant le padding
        lock = is_my_pokemon_locked(self.dict_observation)
        self.action_space_size = len(self.my_team_switches)//16 + lock*1 + (1-lock)*4 #SI lock alors 1 move, sinon 4
        self.action_space_size = self.action_space_size / 9
        self.action_space_size = np.array([self.action_space_size], dtype=np.float32)
        self.my_team_switches = pad_vector(self.my_team_switches,max_len= MAX_LEN_TEAM_SWITCHES)
        self.opponent_active_pokemon = np.array(ObservedPokemon_to_list(self.dict_observation["opponent_active_pokemon"],is_active_pokemon=1,is_opponent_team=1),dtype=np.float32) #à 1 car on veut savoir si il est boosté
        
        #Equipe adverse sans le pokemon actif 
        self.opponent_team = np.array(get_opponent_team_with_all_infos(self.dict_observation,battle),dtype=np.float32)
        self.opponent_team = pad_vector(self.opponent_team ,max_len= MAX_LEN_TEAM_SWITCHES)
        #Fin équipe adverse 
        self.output = np.concatenate([
            self.side_conditions, #Fonctionnel
            self.opponent_side_conditions, #Fonctionnel
            self.weather, #Fonctionnel
            self.active_pokemon, #Fonctionnel
            self.my_team_switches, #Fonctionnel
            self.opponent_active_pokemon, #Fonctionnel
            self.opponent_team, #Fonctionnel
            self.action_space_size #Fonctionnel
        ])

**Agent**

In [23]:
############## DEFINIR L'EMBEDDING ##################
import time
import random
from collections import deque
# Initialiser GenData pour la génération souhaitée (par exemple, génération 4)
gen_data = GenData.from_gen(4)

# Accéder au tableau des types
type_chart = gen_data.type_chart


from poke_env.player.env_player import Gen4EnvSinglePlayer
import numpy as np
import time, random

class Player5Training(Gen4EnvSinglePlayer):
    def __init__(self, model, battle_format="gen4randombattle",max_concurrent_battles=1, **kwargs):
        self._max_concurrent_battles = max_concurrent_battles
        super().__init__(battle_format=battle_format,**kwargs)
        self._init_environment()
        self.model = model
        self.action_space = Discrete(9)
        self.my_last_moves_ids = deque(maxlen=3)
        self.opp_last_moves_ids = deque(maxlen=3)
        self.moves_to_id = moves_to_id  # important si moves_to_id est global

    def _init_environment(self):
            # Hack pour forcer poke-env à prendre en compte le bon nombre de battles
            self._n_battles = self._max_concurrent_battles

    def embed_battle(self, battle):
        # Remplace ça par ta propre obs
        obs = adapted_obs(battle).output
        return obs

    def describe_embedding(self):
        return Box(low=0, high=256, shape=(233,), dtype=np.float32)
    
    def update_last_moves(self, battle):
        if battle.move_turn:
            move = battle.move_turn[0].move
            if move:
                move_id = self.moves_to_id.get(move.id, None)
                if move_id is not None:
                    self.my_last_moves.append(move_id)

        if battle.opponent_active_pokemon and battle.opponent_active_pokemon.moves:
            opp_move = list(battle.opponent_active_pokemon.moves.values())[0]
            if opp_move:
                opp_move_id = self.moves_to_id.get(opp_move.id, None)
                if opp_move_id is not None:
                    self.opp_last_moves.append(opp_move_id)

    
    def _get_boosting_moves_for_stat(self, stat):
        boosting_moves = {
            "atk": ["swordsdance", "bulkup"],
            "spa": ["nastyplot", "calmmind"],
            "def": ["irondefense"],
            "spd": ["amnesia"],
            "spe": ["agility", "rockpolish"],
        }
        return boosting_moves.get(stat, [])
    
    def count_recent_boosts(self, n_turns=3):
        count = 0
        recent_moves = list(self.my_last_moves_ids)[-n_turns:]
        boost_moves = [
            move
            for stat in ["atk", "spa", "def", "spd", "spe"]
            for move in self._get_boosting_moves_for_stat(stat)
        ]

        for move_id in recent_moves:
            move_name = next((k for k, v in self.moves_to_id.items() if v == move_id), None)
            if move_name in boost_moves:
                count += 1

        return count

    def calc_reward(self, last_battle, current_battle):
        reward = self.reward_computing_helper(
            current_battle, fainted_value=2.0, hp_value=1.0, victory_value=30.0
        )

        my_poke = current_battle.active_pokemon
        opp_poke = current_battle.opponent_active_pokemon

        # Derniers coups (ids entiers)
        last_move_id = self.my_last_moves_ids[-1] if self.my_last_moves_ids else None
        prev_move_id = self.my_last_moves_ids[-2] if len(self.my_last_moves_ids) >= 2 else None
        last_opp_move_id = self.opp_last_moves_ids[-1] if self.opp_last_moves_ids else None

        # ========== Double Protect pénalisé ==========
        protect_id = self.moves_to_id.get("protect", 182 + 1)
        if last_move_id == protect_id and prev_move_id == protect_id:
            if opp_poke.status in ["psn", "tox", "brn"]:
                if opp_poke.current_hp < opp_poke.max_hp:
                    reward += 0.5
                else:
                    reward -= 2.0
            else:
                reward -= 3.0

        # ========== Bonus si Abri contre un switch adverse ==========
        if last_move_id == protect_id and current_battle.opponent_switch:
            reward += 2.0

        # ========== Pénalité si attaque sur immunité ==========
        if last_move_id is not None:
            move_name = [k for k, v in self.moves_to_id.items() if v == last_move_id]
            if move_name:
                move_obj = self._get_move_by_id(move_name[0], current_battle)
                if move_obj and move_obj.base_power > 0:
                    effectiveness = move_obj.type.damage_multiplier(opp_poke.type_1, opp_poke.type_2)
                    if effectiveness == 0:
                        if move_obj.base_power >= 60:  # équiv. à last_move_damage >= 0.3
                            reward += 5.0  # anticipation (ex: double switch)
                        else:
                            reward -= 4.0

        # ========== Pénalité si boost sur stat déjà à +6 ==========
        for stat, boost in my_poke.boosts.items():
            if boost >= 6:
                for move_name, move_id in self.moves_to_id.items():
                    if move_id == last_move_id and move_name in self._get_boosting_moves_for_stat(stat):
                        reward -= 2.0
                        break

        # ========== Surboost suicidaire : faint juste après 2 boosts ==========
        if my_poke.fainted:
            nb_boosts = self.count_recent_boosts(n_turns=3)
            if nb_boosts >= 2:
                factor = 3 if nb_boosts >= 3 else 2
                reward -= factor * 2.5

        # ========== Pose de pièges ==========
        turn = current_battle.turn
        stealthrock_id = self.moves_to_id.get("stealthrock", 446 + 1)
        spikes_id = self.moves_to_id.get("spikes", 191 + 1)

        if last_move_id == stealthrock_id:
            if current_battle.fields["stealthrock"]:
                reward -= 2.0
            else:
                reward += 4.0 if turn <= 15 else 2.0

        if last_move_id == spikes_id:
            n_spikes = current_battle.fields.get("spikes", 0)
            if n_spikes >= 3:
                reward -= 2.0
            else:
                reward += 4.0 if turn <= 15 else 2.0

        return reward


    def action_to_move(self, action, battle):
        return super().action_to_move(action, battle)

In [24]:
from stable_baselines3.dqn.policies import DQNPolicy
from stable_baselines3.common.torch_layers import create_mlp
from torch import optim


class CustomDQNPolicySGD(DQNPolicy):
    def __init__(self, observation_space, action_space, lr_schedule, **kwargs):
        kwargs["net_arch"] = [512,256,128,64,64,32]
        super().__init__(observation_space, action_space, lr_schedule, **kwargs)
    def make_optimizers(self) -> None:
        # Redéfinir l'optimiseur ici
        self.optimizer = optim.SGD(
            self.parameters(),
            lr=self.lr_schedule(1),  # learning_rate peut être une fonction
            momentum=0.9,            # optionnel
            weight_decay=0.0         # optionnel
        )

In [25]:
from stable_baselines3.common.vec_env import DummyVecEnv

# Réinstancier l'env
opponent = RandomPlayer(battle_format="gen4randombattle")

train_env_raw = Player5Training(
    model = None,
    battle_format="gen4randombattle",
    opponent=opponent,
    max_concurrent_battles=10
)

train_env = DummyVecEnv([lambda: train_env_raw])  # Redéfini bien après reset

In [26]:
from stable_baselines3 import DQN
#env = train_env

model = DQN(
    policy=CustomDQNPolicySGD, #A la base : CustomDQNPolicy
    env=train_env,
    learning_rate=2.5e-4,
    buffer_size=10000,
    learning_starts=1000,
    batch_size=32,
    gamma=0.5,
    train_freq=8,
    target_update_interval=1,
    exploration_fraction=1.0,
    exploration_final_eps=0.05,
    verbose=1,
    device="mps",
    _init_setup_model=True #intialement à False quand on prenait des envs custom
)

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

Using mps device


In [28]:
train_env.reset()
model.learn(total_timesteps=100)